In [1]:
# todo: check the model cfg: augmentations, policy

In [1]:
from tqdm import tqdm
import numpy as np

import torch
from torch import nn
from utils import parse_cfg, iou_vectorized
from darknet import Darknet

In [2]:
darknet = Darknet('cfg/yolov3_test_todel.cfg')

we also can remove bias due to bn
make_layers returns net_info as well. check whether it"s necessary
shortcut is using output[i-1] instead of x check whether works with x
NOTE THAT CONV BEFORE YOLO USES (num_classes filters) * num_anch
changing predictions in the nms loop make sure that it is not used later
not adding +1 in nms


In [3]:
weight_file = 'weights/yolov3.weights'
darknet.load_weights(weight_file)

In [4]:
from matplotlib import pyplot as plt
import cv2

plt.figure()
img = cv2.imread('dog-cycle-car.png')
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
print(img.shape)
img = cv2.resize(img, (416, 416)) / 255
plt.imshow(img)
plt.show()
img = img.transpose((2, 0, 1))
img = torch.from_numpy(img).float()
img = img.unsqueeze(0)
print(img.shape)
pred = darknet.forward(img, 'cpu')
pred

(452, 602, 3)


<Figure size 640x480 with 1 Axes>

torch.Size([1, 3, 416, 416])


/miniconda3/envs/main/lib/python3.5/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/miniconda3/envs/main/lib/python3.5/site-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


tensor([[100.0774, 378.8565,  16.0677,  29.6695,   0.5525,   0.8951,  58.0000],
        [283.5150, 112.5433, 103.1037, 218.4937,   1.0000,   0.9941,  16.0000],
        [227.8112, 152.9433, 124.7233, 234.4098,   0.6576,   0.9671,  16.0000],
        [216.5894, 169.8118, 241.9540, 201.7436,   0.9944,   0.9987,   1.0000],
        [138.1751,  98.3323,  59.4538,  46.9861,   0.8167,   0.9892,   1.0000],
        [121.8502, 110.1867,  59.8466,  43.6617,   0.7452,   0.9883,   1.0000],
        [ 92.1101, 316.0642, 109.4271,  48.6384,   1.0000,   0.8345,   7.0000],
        [ 66.8319, 347.8462, 110.6174,  56.3349,   0.9997,   0.7319,   7.0000],
        [123.9189, 289.6150, 109.5699,  56.0721,   0.9931,   0.7168,   7.0000]],
       grad_fn=<CatBackward>)

In [14]:
pred

tensor([[100.1483, 378.8937,  17.0948,  32.8473,   0.5465,   0.8928,  58.0000],
        [283.4834, 112.6535, 103.2805, 223.5198,   1.0000,   0.9943,  16.0000],
        [227.7840, 153.0189, 124.9373, 239.8021,   0.6575,   0.9681,  16.0000],
        [216.5429, 169.9060, 242.3690, 206.3844,   0.9944,   0.9987,   1.0000],
        [138.0303,  98.3391,  57.1368,  48.1702,   0.8130,   0.9888,   1.0000],
        [121.7023, 110.1922,  57.5143,  44.7621,   0.7405,   0.9878,   1.0000],
        [ 91.9664, 315.8874, 110.8051,  48.2153,   1.0000,   0.8356,   7.0000],
        [ 66.7268, 347.6610, 112.0103,  55.8449,   0.9997,   0.7335,   7.0000],
        [123.7692, 289.5396, 110.9496,  55.5843,   0.9931,   0.7184,   7.0000]],
       grad_fn=<CatBackward>)

In [6]:
# todo nms
x = torch.randn((2, 3, 416, 416))
device = torch.device('cpu:0')
predictions = darknet.forward(x, device)
predictions[:, :, 4] *= 1000 # to be removed. just for testing
predictions = pred
objectness_thres = 0.5
nms_thres = 0.4
classes = 80 # darknet.layers_list[-1][0].classes

/miniconda3/envs/main/lib/python3.5/site-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.Upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))
/miniconda3/envs/main/lib/python3.5/site-packages/torch/nn/functional.py:2423: UserWarning: Default upsampling behavior when mode=bilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.
  "See the documentation of nn.Upsample for details.".format(mode))


here


In [6]:
bboxes1 = torch.Tensor([(6, 5, 4, 2), (3, 6, 4, 2), (8, 4, 2, 8), (11, 7, 2, 2), (8, 0, 4, 2)]).float()
bboxes2 = bboxes1[0].unsqueeze(0)

iou_vectorized(bboxes1, bboxes2)

tensor([[1.0000],
        [0.0667],
        [0.0909],
        [0.0000],
        [0.0000]])

In [7]:
# Non-max Suppression:
# 1. Filter out predictions with low objectness score
# 2. 
# objectiveness filtering:
# replace all boxes with predicted probability lower than objectness_thres 
# with zeros: calculate 0/1 mask and apply it to the prediction tensor
# Note: '>' returns Byte but '*' needs Float unsqueezed back to 3D
# objectness_mask = (predictions[:, :, 4] > objectness_thres).float().unsqueeze(2)
# predictions = predictions * objectness_mask

# # 


# image_pred = predictions[0]
# image_pred.shape

In [21]:
print('changing predictions in the nms loop make sure that it is not used later')
print('not adding +1')

detections = [None] * len(predictions)

for i, prediction in enumerate(predictions):
#     print(prediction.shape)
    objectness_mask = (prediction[:, 4] > objectness_thres)#.float().unsqueeze(1)
    prediction = prediction[objectness_mask]
    
    # if no object on an image found, continue with the next image
    if prediction.size(0) == 0:
        continue
        
    print(prediction.shape)
    pred_score, pred_classes = torch.max(prediction[:, 5:5+classes], dim=1, keepdim=True)
#     print(pred_classes.shape, pred_score.shape)
    # detections: (cx, cy, w, h, obj_score, top_class_score, top_class_idx)
    prediction = torch.cat((prediction[:, :5], pred_score.float(), pred_classes.float()), dim=1)
#     print(prediction.shape)
    unique_classes = pred_classes.unique().float()
#     print(unique_classes.shape)
    
    detections_after_nms = []
    
    for cls in tqdm(unique_classes):
        prediction_4_cls = prediction[prediction[:, 6] == cls]
        sort_pred_idxs = torch.sort(prediction_4_cls[:, 4], descending=True)[1]
        prediction_4_cls = prediction_4_cls[sort_pred_idxs]          
        
        while len(prediction_4_cls) > 0:
            detections_after_nms.append(prediction_4_cls[0].unsqueeze(0))
            print(prediction_4_cls[0].unsqueeze(0))
            
            if len(prediction_4_cls) == 1:
                break
            
#             print('prediction_4_cls[0, :5].shape, prediction_4_cls[1:, :5].shape')
#             print(prediction_4_cls[0, :5].shape, prediction_4_cls[1:, :5].shape)
            ious = iou_vectorized(prediction_4_cls[0, :5].unsqueeze(0), prediction_4_cls[1:, :5])
            ious = ious.reshape(-1)
#             print(iou.shape)
#             print('iou < nms_thres')
#             print(iou < nms_thres)
#             print(iou.shape)
            prediction_4_cls = prediction_4_cls[1:][ious < nms_thres]
            
        # append to detections

100%|██████████| 4/4 [00:00<00:00, 400.40it/s]

changing predictions in the nms loop make sure that it is not used later
torch.Size([22, 85])
tensor([[100.2575, 378.8981,  16.0283,  29.6745,   0.5537,   0.8899,  58.0000]],
       grad_fn=<UnsqueezeBackward0>)
tensor([[283.3438, 112.5091, 102.9544, 212.8765,   1.0000,   0.9941,  16.0000]],
       grad_fn=<UnsqueezeBackward0>)
tensor([[227.6670, 152.9198, 124.5427, 228.3836,   0.6641,   0.9669,  16.0000]],
       grad_fn=<UnsqueezeBackward0>)
tensor([[216.3377, 169.7828, 241.6037, 196.5571,   0.9946,   0.9987,   1.0000]],
       grad_fn=<UnsqueezeBackward0>)
tensor([[138.1874,  98.4530,  57.0273,  47.3960,   0.8196,   0.9886,   1.0000]],
       grad_fn=<UnsqueezeBackward0>)
tensor([[121.8628, 110.2799,  57.4041,  44.0426,   0.7489,   0.9876,   1.0000]],
       grad_fn=<UnsqueezeBackward0>)
tensor([[240.0890, 207.0442, 364.6438, 333.7238,   0.5685,   0.8301,   1.0000]],
       grad_fn=<UnsqueezeBackward0>)
tensor([[ 92.0773, 315.9416, 114.3170,  49.7113,   0.9999,   0.8397,   7.0000]],

In [26]:
torch.cat(detections_after_nms).shape

torch.Size([10, 7])

In [5]:
sum(p.numel() for p in darknet.parameters() if p.requires_grad)

61949149

In [22]:
bn.runn.data

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.])

In [ ]:
nn.BatchNorm2d()